In [1]:
# ✅ STEP 1: Install Required Packages
!pip install -q transformers sentence-transformers faiss-cpu

# ✅ STEP 2: Import Libraries
import pandas as pd
import re
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

# ✅ STEP 3: Load Datasets
recipes_df = pd.read_csv("/content/70000_recipes_nutrients.csv")
profile_df = pd.read_csv("/content/health_age_data_70000_synthetic.csv")

# ✅ STEP 4: Clean Ingredients
def clean_ingredients(text):
    if pd.isna(text): return ""
    text = re.sub(r"[^a-zA-Z0-9, ]", "", text)
    return ', '.join([w.strip().lower() for w in text.split(',') if w.strip()])

# ✅ STEP 5: Find Similar Dishes Using Sentence Embeddings
def get_similar_dishes(dish_name, top_k=3):
    model = SentenceTransformer("all-MiniLM-L6-v2")
    names = recipes_df["recipe_name"].astype(str).tolist()
    embeddings = model.encode(names)
    target_emb = model.encode([dish_name])
    scores = cosine_similarity(target_emb, embeddings)[0]
    top_indices = scores.argsort()[::-1][:top_k]
    return [names[i] for i in top_indices]

# ✅ STEP 6: Nutrient Targets
def get_user_nutrient_goals(age):
    return {
        "Daily Calorie Target": 2500 if age < 50 else 2200,
        "Protein": 200,
        "Carbohydrates": 250,
        "Fat": 70,
        "Fiber": 30
    }

# ✅ STEP 7: Health Rules
def build_health_rules():
    return {
        "diabetes": {"avoid": ["sugar", "ghee", "salt"], "recommend": ["cinnamon", "olive oil", "spinach"]},
        "heart disease": {"avoid": ["butter", "cream", "red meat"], "recommend": ["garlic", "spinach", "flaxseeds"]},
        "obesity": {"avoid": ["soda", "fries", "white bread"], "recommend": ["vegetables", "oats", "lean protein"]},
        "hypertension": {"avoid": ["salt", "processed meat"], "recommend": ["leafy greens", "olive oil"]},
        "cholesterol": {"avoid": ["egg yolk", "red meat", "saturated fat"], "recommend": ["chia seeds", "walnuts", "whole grains"]}
    }

health_rules = build_health_rules()

approved_additions = [
    "turmeric", "olive oil", "cinnamon", "spinach", "broccoli", "kale",
    "bell peppers", "mushrooms", "zucchini", "flaxseeds", "chia seeds", "whole grain pasta"
]

# ✅ STEP 8: Falcon Prompt Builder
def build_prompt_falcon(dish_name, age, disease, ingredients, avoid, recommend, targets):
    cleaned = clean_ingredients(ingredients)
    cleaned_list = [i.strip() for i in cleaned.split(',') if i.strip()]
    bullet_ing = "\n".join(f"- {item}" for item in cleaned_list)

    return f"""
You are a professional nutritionist.

A {age}-year-old patient with {disease} eats a dish called "{dish_name}".

Ingredients:
{bullet_ing}

Nutrition Targets:
- Calories: {targets["Daily Calorie Target"]}
- Protein: {targets["Protein"]}g
- Carbs: {targets["Carbohydrates"]}g
- Fat: {targets["Fat"]}g
- Fiber: {targets["Fiber"]}g

Avoid: {', '.join(avoid)}
Only recommend additions from: {', '.join(approved_additions)}

Write a short paragraph:
- List harmful ingredients to remove and why.
- List healthy additions and why.
- Explain how the changes improve the dish for the given condition.
"""

# ✅ STEP 9: Load Falcon-RW-1B
falcon_tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-rw-1b")
falcon_model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-rw-1b")
falcon_model.eval().to("cuda" if torch.cuda.is_available() else "cpu")

# ✅ STEP 10: Text Generation with Falcon
def falcon_llm(prompt, max_tokens=256):
    inputs = falcon_tokenizer(prompt, return_tensors="pt").to(falcon_model.device)
    with torch.no_grad():
        outputs = falcon_model.generate(**inputs, max_new_tokens=max_tokens)
    return falcon_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

# ✅ STEP 11: Final Output Pipeline
def generate_output_falcon(dish_name, age, disease):
    print(f"🔍 Finding best match for: {dish_name}")
    match = get_similar_dishes(dish_name)[0]
    row = recipes_df[recipes_df["recipe_name"].str.lower() == match.lower()]
    if row.empty:
        return f"❌ Recipe '{match}' not found"

    raw_ing = row.iloc[0]["ingredients"]
    targets = get_user_nutrient_goals(age)
    disease_lower = disease.strip().lower()
    matched_key = next((key for key in health_rules if key in disease_lower), None)
    if matched_key is None:
        return f"❌ No health rules found for any condition in: '{disease}'"

    avoid = health_rules[matched_key]["avoid"]
    recommend = health_rules[matched_key]["recommend"]

    prompt = build_prompt_falcon(
        dish_name=match,
        age=age,
        disease=disease,
        ingredients=raw_ing,
        avoid=avoid,
        recommend=recommend,
        targets=targets
    )

    print("\n📨 Prompt Sent:\n", prompt)
    result = falcon_llm(prompt)
    print("\n🤖 Falcon-RW-1B Response:\n", result)
    return result

# ✅ TEST
generate_output_falcon("cake", 45, "diabetes, hypertension, heart disease")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.1 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



📨 Prompt Sent:
 
You are a professional nutritionist.

A 45-year-old patient with diabetes, hypertension, heart disease eats a dish called "king cake".

Ingredients:
- active dry yeast
- white sugar
- warm milk 110 degrees f45 degrees c
- butter
- egg yolks
- allpurpose flour
- salt
- ground nutmeg
- grated lemon zest
- cream cheese
- confectioners sugar
- confectioners sugar
- lemon juice
- milk
- multicolored candy sprinkles

Nutrition Targets:
- Calories: 2500
- Protein: 200g
- Carbs: 250g
- Fat: 70g
- Fiber: 30g

Avoid: sugar, ghee, salt
Only recommend additions from: turmeric, olive oil, cinnamon, spinach, broccoli, kale, bell peppers, mushrooms, zucchini, flaxseeds, chia seeds, whole grain pasta

Write a short paragraph:
- List harmful ingredients to remove and why.
- List healthy additions and why.
- Explain how the changes improve the dish for the given condition.


🤖 Falcon-RW-1B Response:
 You are a professional nutritionist.

A 45-year-old patient with diabetes, hypertensio

'You are a professional nutritionist.\n\nA 45-year-old patient with diabetes, hypertension, heart disease eats a dish called "king cake".\n\nIngredients:\n- active dry yeast\n- white sugar\n- warm milk 110 degrees f45 degrees c\n- butter\n- egg yolks\n- allpurpose flour\n- salt\n- ground nutmeg\n- grated lemon zest\n- cream cheese\n- confectioners sugar\n- confectioners sugar\n- lemon juice\n- milk\n- multicolored candy sprinkles\n\nNutrition Targets:\n- Calories: 2500\n- Protein: 200g\n- Carbs: 250g\n- Fat: 70g\n- Fiber: 30g\n\nAvoid: sugar, ghee, salt\nOnly recommend additions from: turmeric, olive oil, cinnamon, spinach, broccoli, kale, bell peppers, mushrooms, zucchini, flaxseeds, chia seeds, whole grain pasta\n\nWrite a short paragraph:\n- List harmful ingredients to remove and why.\n- List healthy additions and why.\n- Explain how the changes improve the dish for the given condition.\n- Explain how the changes improve the dish for the given condition.\n- Explain how the changes i